# Precipitation seasonality and variability

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # NetCDF library
import cartopy  # Plotting libary
import cartopy.crs as ccrs  # Projections
np.set_printoptions(threshold=20)  # avoid to print very large arrays on screen
# The commands below are not important
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Some defaults
plt.rcParams['figure.figsize'] = (14, 5)  # Default plot size

## Precipitation data 

In [ ]:
netcdf = xr.open_dataset('ERA-Int-Monthly-P.nc')

## Seasonality index 

The seasonality index by Walsh and Lawler (1981) is a convenient way to visualize the regions where precipitation tends to be seasonally concentrated:

$$ SI = \frac{1}{P_A}  \sum_{i=1}^{12} \lvert P_i - \frac{P_A}{12} \rvert$$

It can be interpreted as follows:

    < 0.19         Precipitation  spread  throughout  the  year
    0.20-0.39      Precipitation  spread  throughout  the  year,  but  with  a  definite  wetter  season
    0.40-0.59      Rather  seasonal  with  a  short  drier  season
    0.60-0.79      Seasonal
    0.80-0.99      Markedly  seasonal  with  a  long  dry  season
    1.00-1.19      Most  precipitation in 3  months
    > 1.20         Extreme  seasonality,  with  almost  all  precipitation  in  1–2  months

**Compute it:**

In [ ]:
prcp_cycle = netcdf.tp.copy().groupby('time.month').mean(dim='time')
r = prcp_cycle.sum(dim='month')
si = 1 / r * (np.abs(prcp_cycle - r/12.)).sum(dim='month')
si.name = ''

In [ ]:
levels = [0.2, 0.4, 0.6, 0.8, 1, 1.2]
ax = plt.axes(projection=ccrs.Robinson())
si.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=plt.get_cmap('BuPu'), levels=levels) 
plt.title("Walsh and Lawler seasonality index for ERA-Int precipitation 1979-2014")
ax.coastlines();

## Inter-annual variability 

Annual precipitation time serie:

In [ ]:
prcp_a = netcdf.tp.copy().resample(dim='time', freq='AS', how='sum')

Annual average precipitation:

In [ ]:
prcp_a_tot = prcp_a.mean(dim='time')
prcp_a_tot.name = 'm yr-1'

Standard deviation of annual precipitation:

In [ ]:
prcp_a_std = prcp_a.std(dim='time')
prcp_a_std.name = 'm yr-1'

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
prcp_a_std.plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree(), cmap=plt.get_cmap('BuPu'), vmax=1) 
plt.title("Standard deviation of annual precipitation")
ax.coastlines();

**Relation between standard deviation and total precipitation:**

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 5))
ax.scatter(prcp_a_tot, prcp_a_std);
ax.set_ylim([0, 4])
ax.set_xlim([0, 12])
ax.set_xlabel('Total annual precipitation (m yr-1)')
ax.set_ylabel('StdDev annual precipitation (m yr-1)');

The **coefficient of variation** is therefore a better measure of variability:

$$ c_v = \frac{ \sigma}{\mu} $$

In [ ]:
cv = prcp_a_std / prcp_a_tot
cv.name = ''

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
cv.plot(ax=ax, origin='upper', aspect='equal', transform=ccrs.PlateCarree(), cmap=plt.get_cmap('BuPu'), vmax=0.5) 
plt.title("Coefficient of variation of annual precipitation")
ax.coastlines();

**Relation between coefficient of variation and total precipitation:**

In [ ]:
f, ax = plt.subplots(1, figsize=(5, 5))
ax.scatter(prcp_a_tot, cv);
ax.set_ylim([0, 4])
ax.set_xlim([0, 12])
ax.set_xlabel('Total annual precipitation (m yr-1)')
ax.set_ylabel('Coefficient of variation (-)');